In [1]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[midje "1.9.1"])
(use 'clojure.pprint)
(use 'clojure.repl)
(use 'midje.repl)

Run `(doc midje)` for Midje usage.
Run `(doc midje-repl)` for descriptions of Midje repl functions.


# Functional Programming

## Pure Functions: What and Why

1. _Referential Transparency_

   always returns the same result given the same arguments

2. Side-effect free

   can't make any changes that are observable outside the function itself

### Pure Functions Are Referentially Transparent

In [2]:
(fact
    (+ 1 2) => 3)

true

In [3]:
(defn wisdom
  [words]
  (str words ", Daniel-san"))

(fact
    (wisdom "Always bathe on Fridays") => "Always bathe on Fridays, Daniel-san")

true

In [4]:
(defn year-end-evaluation
  []
  (if (> (rand) 0.5)
    "You get a raise!"
    "Better luck next year!"))
(fact
    (year-end-evaluation) => "You get a raise!")


FAIL at (core.clj:7)
Expected:
"You get a raise!"
Actual:
"Better luck next year!"
Diffs: strings have 1 difference (4% similarity)
                expected: "(You get a raise------)!"
                actual:   "(Better luck next year)!"


false

In [5]:
; this function _is_ pure --- given the same text, the count will be the same
(defn analysis
  [text]
  (str "Character count: " (count text)))

; this function is not pure because it is reading from the filesystem
(defn analyze-file
  [filename]
  (analysis (slurp filename)))



(fact
    (analyze-file "resources/text-file.txt") => "Character count: 13")

true

### Pure Functions Have No Side Effects

To perform a side effect is to change the association between a name and its value within a given scope:

```javascript
var haplessObject = {
  emotion: "Carefree!"
};

var evilMutator = function(object){
  object.emotion = "So emo :'(";
}

evilMutator(haplessObject);
haplessObject.emotion;
// => "So emo :'("
```

## Living with Immutable Data

- P. Helland, "Immutability Changes Everything," _ACM Queue_, November--December 2015 [[link]](https://dl.acm.org/citation.cfm?id=2884038)

### Recursion Instead of for/while

```js
var wrestlers = getAlligatorWrestlers();
var totalBites = 0;
var l = wrestlers.length;

for(var i=0; i < l; i++){
  totalBites += wrestlers[i].timesBitten;
}
```

```js
var allPatients = getArkhamPatients();
var analyzedPatients = [];
var l = allPatients.length;

for(var i=0; i < l; i++){
  if(allPatients[i].analyzed){
    analyzedPatients.push(allPatients[i]);
  }
}
```

In [6]:
(def great-baby-name "Rosanthony")
(fact
    great-baby-name => "Rosanthony")

true

In [7]:
(let [great-baby-name "Bloodthunder"]
  (fact
      great-baby-name => "Bloodthunder"))

true

In [8]:
(fact
    great-baby-name => "Rosanthony")

true

In [9]:
(defn sum
    ; arity overloading
    ([vals] (sum vals 0)) 
    ([vals accumulating-total]
     (if (empty? vals)
         accumulating-total
         (sum (rest vals) (+ (first vals) accumulating-total)))))

(fact
    (sum (range 11)) => 55)

true

In [10]:
; clojure *does not* have tail-call optimization
(sum (range 1000000))

StackOverflowError   clojure.lang.LongRange.next (LongRange.java:142)


class java.lang.StackOverflowError: 

In [11]:
(doc recur)

-------------------------
recur
  (recur exprs*)
Special Form
  Evaluates the exprs in order, then, in parallel, rebinds
  the bindings of the recursion point to the values of the exprs.
  Execution then jumps back to the recursion point, a loop or fn method.

  Please see http://clojure.org/special_forms#recur


In [12]:
(defn sum
    ([vals]
     (sum vals 0))
    ([vals accumulating-total]
     (if (empty? vals)
         accumulating-total
         (recur (rest vals) (+ (first vals) accumulating-total)))))

#'user/sum

In [13]:
(fact
    (sum (range 11)) => 55)

true

In [14]:
(sum (range 1000000))

499999500000

*NOTE:* Clojure’s immutable data structures are implemented using _structural sharing_. This keeps the program from thrashing due to excessive GC. See [Understanding Clojure's Persistent Vectors](https://hypirion.com/musings/understanding-persistent-vector-pt-1) for more info.

### Function Composition Instead of Attribute Mutation

```ruby
class GlamourShotCaption
  attr_reader :text
  def initialize(text)
    @text = text
    clean!
  end

  private
  def clean!
    text.trim!
    text.gsub!(/lol/, "LOL")
  end
end

best = GlamourShotCaption.new("My boa constrictor is so sassy lol!  ")
best.text
```

In [15]:
(require '[clojure.string :as s])
(defn clean
  [text]
  (s/replace (s/trim text) #"lol" "LOL"))


(fact
    (clean "My boa constrictor is so sassy lol!  ") => "My boa constrictor is so sassy LOL!")


true

## Exercises

1. Fibonacci number

In [16]:
(defn fib [n]
    (cond
        (= n 0) 0
        (= n 1) 1
        :else (+ (fib (- n 1)) (fib (- n 2)))))
    

(facts "about fib"
       (fact (fib 3) => 2)
       (fact (fib 9) => 34))


true

In [17]:
(defn recursive-fib
    ([n]
     (recursive-fib n 0 1))
    ([n a b]
    (cond
        (= n 0) a
        (= n 1) b
        :else (recur (- n 1) b (+ a b)))))

(facts "about fib"
       (fact (recursive-fib 3) => 2)
       (fact (recursive-fib 9) => 34))

true

In [18]:
(time (recursive-fib 20))

"Elapsed time: 0.122813 msecs"


6765

In [19]:
(time (fib 20))

"Elapsed time: 6.13163 msecs"


6765

In [20]:
(def m-fib (memoize fib))

#'user/m-fib

In [21]:
(time (m-fib 20))

"Elapsed time: 0.739654 msecs"


6765